In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/My Drive'

/content/drive/My Drive


In [ ]:
!nvidia-smi

Wed Jul 22 16:33:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install Required Libraries from PyPI
!pip install face-detection
!pip install tqdm

In [ ]:
# Import Required Libraries
import time
import math
import os
import numpy as np
import cv2
import face_detection 
from sklearn.cluster import DBSCAN
from keras.models import load_model
from keras.applications.resnet50 import preprocess_input
import tqdm
from google.colab.patches import cv2_imshow

Using TensorFlow backend.


In [ ]:
confid = 0.5
thresh = 0.5
angle_factor = 0.8
H_zoom_factor = 1.2

In [ ]:
def dist(c1, c2):
    return ((c1[0] - c2[0]) ** 2 + (c1[1] - c2[1]) ** 2) ** 0.5

def T2S(T):
    S = abs(T/((1+T**2)**0.5))
    return S

def T2C(T):
    C = abs(1/((1+T**2)**0.5))
    return C

def isclose(p1,p2):

    c_d = dist(p1[2], p2[2])
    if(p1[1]<p2[1]):
        a_w = p1[0]
        a_h = p1[1]
    else:
        a_w = p2[0]
        a_h = p2[1]

    T = 0
    try:
        T=(p2[2][1]-p1[2][1])/(p2[2][0]-p1[2][0])
    except ZeroDivisionError:
        T = 1.633123935319537e+16
    S = T2S(T)
    C = T2C(T)
    d_hor = C*c_d
    d_ver = S*c_d
    vc_calib_hor = a_w*1.3
    vc_calib_ver = a_h*0.4*angle_factor
    c_calib_hor = a_w *1.7
    c_calib_ver = a_h*0.2*angle_factor
    # print(p1[2], p2[2],(vc_calib_hor,d_hor),(vc_calib_ver,d_ver))
    if (0<d_hor<vc_calib_hor and 0<d_ver<vc_calib_ver):
        return 1
    elif 0<d_hor<c_calib_hor and 0<d_ver<c_calib_ver:
        return 2
    else:
        return 0

In [ ]:
# Path to the Working Environment

# If using Google Colab (If on a Local Environment, no path required => set BASE_PATH  = "")
BASE_PATH = "Social_Distancing_with_AI/"

# Path to Input Video File in the BASE_PATH
FILE_PATH = "test_video.mp4"

In [ ]:
# Initialize a Face Detector 
# Confidence Threshold can be Adjusted, Greater values would Detect only Clear Faces
detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.8, nms_iou_threshold=.4)

In [ ]:
# Load Pretrained Face Mask Classfier (Keras Model)
mask_classifier = load_model("Social_Distancing_with_AI/Models/ResNet50_Classifier.h5")

In [ ]:
# Set the Safe Distance in Pixel Units (Minimum Distance Expected to be Maintained between People)
# This Parameter would Affect the Results, Adjust according to the Footage captured by CCTV Camera 
threshold_distance = 72  # Try with different Values before Finalizing

In [ ]:
##################################### Analyze the Video ################################################

# Load YOLOv3
net = cv2.dnn.readNet(BASE_PATH+"Models/"+"yolov3-spp.weights", BASE_PATH+"Models/"+"yolov3-spp.cfg")

# Load COCO Classes
classes = []
with open(BASE_PATH+"Models/"+"coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Fetch Video Properties
cap = cv2.VideoCapture(BASE_PATH + FILE_PATH )
fps = cap.get(cv2.CAP_PROP_FPS)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# Create Directory for Storing Results (Make sure it doesn't already exists !)
os.mkdir(BASE_PATH+"Results")
os.mkdir(BASE_PATH+"Results/Extracted_Faces")
os.mkdir(BASE_PATH+"Results/Extracted_Persons")
os.mkdir(BASE_PATH+"Results/Frames")

# Initialize Output Video Stream
out_stream = cv2.VideoWriter(
    BASE_PATH+'Results/test.mp4',
    cv2.VideoWriter_fourcc('X','V','I','D'),
    fps,
    (int(width),int(height)))

print("Processing Frames :")
for frame in tqdm.notebook.tqdm(range(int(n_frames))):
    
    # Capture Frame-by-Frame
    ret, img = cap.read()

    # Check EOF
    if ret == False:
        break;

    # Get Frame Dimentions
    height, width, channels = img.shape

    # Detect Objects in the Frame with YOLOv3
    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (608, 608), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    
    # Store Detected Objects with Labels, Bounding_Boxes and their Confidences
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.05:
                
                # Get Center, Height and Width of the Box
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Topleft Co-ordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Initialize empty lists for storing Bounding Boxes of People and their Faces
    persons = []
    masked_faces = []
    unmasked_faces = []

    # Work on Detected Persons in the Frame
    for i in range(len(boxes)):
        if i in indexes:

            box = np.array(boxes[i])
            box = np.where(box<0,0,box)
            (x, y, w, h) = box

            label = str(classes[class_ids[i]])

            if label=='person':

                persons.append([x,y,w,h])
                
                # Save Image of Cropped Person (If not required, comment the command below)
                # cv2.imwrite(BASE_PATH + "Results3/Extracted_Persons/"+str(frame)
                #             +"_"+str(len(persons))+".jpg",
                #             img[y:y+h,x:x+w])

                # Detect Face in the Person
                person_rgb = img[y:y+h,x:x+w,::-1]   # Crop & BGR to RGB
                detections = detector.detect(person_rgb)

                # If a Face is Detected
                if detections.shape[0] > 0:

                  detection = np.array(detections[0])
                  detection = np.where(detection<0,0,detection)

                  # Calculating Co-ordinates of the Detected Face
                  x1 = x + int(detection[0])
                  x2 = x + int(detection[2])
                  y1 = y + int(detection[1])
                  y2 = y + int(detection[3])

                  try :

                    # Crop & BGR to RGB
                    face_rgb = img[y1:y2,x1:x2,::-1]   

                    # Preprocess the Image
                    face_arr = cv2.resize(face_rgb, (224, 224), interpolation=cv2.INTER_NEAREST)
                    face_arr = np.expand_dims(face_arr, axis=0)
                    face_arr = preprocess_input(face_arr)

                    # Predict if the Face is Masked or Not
                    score = mask_classifier.predict(face_arr)

                    # Determine and store Results
                    if score[0][0]<0.10:
                      masked_faces.append([x1,y1,x2,y2])
                    else:
                      unmasked_faces.append([x1,y1,x2,y2])

                    # Save Image of Cropped Face (If not required, comment the command below)
                    # cv2.imwrite(BASE_PATH + "Results1/Extracted_Faces/"+str(frame)
                    #             +"_"+str(len(persons))+".jpg",
                    #             img[y1:y2,x1:x2])

                  except:
                    continue
    
    # Calculate Coordinates of People Detected and find Clusters using DBSCAN
    person_coordinates = []

    for p in range(len(persons)):
      person_coordinates.append((persons[p][0]+int(persons[p][2]/2),persons[p][1]+int(persons[p][3]/2)))

    clustering = DBSCAN(eps=threshold_distance,min_samples=2).fit(person_coordinates)
    isSafe = clustering.labels_

    # Count 
    person_count = len(persons)
    masked_face_count = len(masked_faces)
    unmasked_face_count = len(unmasked_faces)
    safe_count = np.sum((isSafe==-1)*1)
    unsafe_count = person_count - safe_count

    # Show Clusters using Red Lines
    arg_sorted = np.argsort(isSafe)

    for i in range(1,person_count):

      if isSafe[arg_sorted[i]]!=-1 and isSafe[arg_sorted[i]]==isSafe[arg_sorted[i-1]]:
        cv2.line(img,person_coordinates[arg_sorted[i]],person_coordinates[arg_sorted[i-1]],(0,0,255),2)

    # Put Bounding Boxes on People in the Frame
    for p in range(person_count):

      a,b,c,d = persons[p]

      # Green if Safe, Red if UnSafe
      if isSafe[p]==-1:
        cv2.rectangle(img, (a, b), (a + c, b + d), (0,255,0), 2)
      else:
        cv2.rectangle(img, (a, b), (a + c, b + d), (0,0,255), 2)

    # Put Bounding Boxes on Faces in the Frame
    # Green if Safe, Red if UnSafe
    for f in range(masked_face_count):

      a,b,c,d = masked_faces[f]
      cv2.rectangle(img, (a, b), (c,d), (0,255,0), 2)

    for f in range(unmasked_face_count):

      a,b,c,d = unmasked_faces[f]
      cv2.rectangle(img, (a, b), (c,d), (0,0,255), 2)

    # Show Monitoring Status in a Black Box at the Top
    cv2.rectangle(img,(0,0),(width,50),(0,0,0),-1)
    cv2.rectangle(img,(1,1),(width-1,50),(255,255,255),2)

    xpos = 15

    string = "Total People = "+str(person_count)
    cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]

    string = " ( "+str(safe_count) + " Safe "
    cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]

    string = str(unsafe_count)+ " Unsafe ) "
    cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]
    
    string = "( " +str(masked_face_count)+" Masked "+str(unmasked_face_count)+" Unmasked "+\
             str(person_count-masked_face_count-unmasked_face_count)+" Unknown )"
    cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)

    # Write Frame to the Output File
    out_stream.write(img)

    # Save the Frame in frame_no.png format (If not required, comment the command below)
    # cv2.imwrite(BASE_PATH+"Results1/Frames/"+str(frame)+".jpg",img)

    # Use if you want to see Results Frame by Frame
    cv2_imshow(img)

    # Exit on Pressing Q Key
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release Streams
out_stream.release()
cap.release()
cv2.destroyAllWindows()

# Good to Go!
print("Done !")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
##################################### Analyze the Video ################################################

# Load YOLOv3
net = cv2.dnn.readNet(BASE_PATH+"Models/"+"yolov3.weights", BASE_PATH+"Models/"+"yolov3.cfg")

# Load COCO Classes
LABELS  = []
with open(BASE_PATH+"Models/"+"coco.names", "r") as f:
    LABELS = [line.strip() for line in f.readlines()]

np.random.seed(42)

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
FR=0
# Fetch Video Properties
vs = cv2.VideoCapture(BASE_PATH + FILE_PATH )
writer = None
(W, H) = (None, None)

fl = 0
q = 0
# Create Directory for Storing Results (Make sure it doesn't already exists !)

# Initialize Output Video Stream
# out_stream = cv2.VideoWriter(
#     BASE_PATH+'Results/test.mp4',
#     cv2.VideoWriter_fourcc('X','V','I','D'),
#     fps,
#     (int(width),int(height)))

print("Processing Frames :")
while True:
  
    # Capture Frame-by-Frame
    (grabbed, frame) = vs.read()

    # Check EOF
    if not grabbed:
        break

    if W is None or H is None:
        (H, W) = frame.shape[:2]
        FW=W
        if(W<1075):
            FW = 1075
        FR = np.zeros((H+210,FW,3), np.uint8)

        col = (255,255,255)
        FH = H + 210
    FR[:] = col

    # Detect Objects in the Frame with YOLOv3
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    outs = net.forward(output_layers)
    end = time.time()

    classIDs  = []
    confidences = []
    boxes = []
    
    # Store Detected Objects with Labels, Bounding_Boxes and their Confidences
    for out in outs:

        for detection in out:


            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if LABELS[classID] == "person":
                if confidence > confid:
                
                    # Get Center, Height and Width of the Box
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    
                
                    # Topleft Co-ordinates
                    # x = int(center_x - w / 2)
                    # y = int(center_y - h / 2)

                    boxes.append([x, y,int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confid,thresh)

    # Initialize empty lists for storing Bounding Boxes of People and their Faces
    

    # Work on Detected Persons in the Frame
    if len(idxs) > 0:
        persons = []
        masked_faces = []
        unmasked_faces = []
        status = []
        idf = idxs.flatten()
        close_pair = []
        s_close_pair = []
        center = []
        co_info = []

        for i in idf:

            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
    
            cen = [int(x + w / 2), int(y + h / 2)]
            center.append(cen)
            cv2.circle(frame, tuple(cen),1,(0,0,0),1)
            co_info.append([w, h, cen])

            status.append(0)
            persons.append([x,y,w,h])
                
                # Save Image of Cropped Person (If not required, comment the command below)
                # cv2.imwrite(BASE_PATH + "Results3/Extracted_Persons/"+str(frame)
                #             +"_"+str(len(persons))+".jpg",
                #             img[y:y+h,x:x+w])

                # Detect Face in the Person
            person_rgb = frame[y:y+h,x:x+w,::-1]   # Crop & BGR to RGB
            detections = detector.detect(person_rgb)

                # If a Face is Detected
            if detections.shape[0] > 0:

                detection = np.array(detections[0])
                detection = np.where(detection<0,0,detection)

                # Calculating Co-ordinates of the Detected Face
                x1 = x + int(detection[0])
                x2 = x + int(detection[2])
                y1 = y + int(detection[1])
                y2 = y + int(detection[3])

                try :


                    # Crop & BGR to RGB
                    face_rgb = frame[y1:y2,x1:x2,::-1]   

                    # Preprocess the Image
                    face_arr = cv2.resize(face_rgb, (224, 224), interpolation=cv2.INTER_NEAREST)
                    face_arr = np.expand_dims(face_arr, axis=0)
                    face_arr = preprocess_input(face_arr)

                    # Predict if the Face is Masked or Not
                    score = mask_classifier.predict(face_arr)

                    # Determine and store Results
                    if score[0][0]<0.10:
                        masked_faces.append([x1,y1,x2,y2])
                    else:
                        unmasked_faces.append([x1,y1,x2,y2])

                    # Save Image of Cropped Face (If not required, comment the command below)
                    # cv2.imwrite(BASE_PATH + "Results1/Extracted_Faces/"+str(frame)
                    #             +"_"+str(len(persons))+".jpg",
                    #             img[y1:y2,x1:x2])

                except:
                    continue
        masked_face_count = len(masked_faces)
        unmasked_face_count = len(unmasked_faces)
        for i in range(len(center)):
            for j in range(len(center)):
    
                g = isclose(co_info[i],co_info[j])

                if g == 1:

                    close_pair.append([center[i], center[j]])
                    status[i] = 1
                    status[j] = 1
                elif g == 2:
                    s_close_pair.append([center[i], center[j]])
                    if status[i] != 1:
                        status[i] = 2
                    if status[j] != 1:
                        status[j] = 2
        total_p = len(center)
        low_risk_p = status.count(2)
        high_risk_p = status.count(1)
        safe_p = status.count(0)
        kk = 0                       
        for i in idf:
            cv2.line(FR,(0,H+1),(FW,H+1),(0,0,0),2)
            cv2.putText(FR, "Social Distancing Analyser and Mask Monitoring wrt. COVID-19", (210, H+60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
            cv2.rectangle(FR, (20, H+80), (510, H+180), (100, 100, 100), 2)
            cv2.putText(FR, "Connecting lines shows closeness among people. ", (30, H+100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (100, 100, 0), 2)
            cv2.putText(FR, "-- YELLOW: CLOSE", (50, H+90+40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 170, 170), 2)
            cv2.putText(FR, "--    RED: VERY CLOSE", (50, H+40+110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            # cv2.putText(frame, "--    PINK: Pathway for Calibration", (50, 150),
            #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (180,105,255), 1)

            cv2.rectangle(FR, (535, H+80), (1250, H+140+40), (100, 100, 100), 2)
            cv2.putText(FR, "Bounding box shows the level of risk to the person and Mask Monitoring", (545, H+100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (100, 100, 0), 2)
            cv2.putText(FR, "-- LIGHT GREEN: SAFE", (565,  H+90+40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(FR, "-- Green: MASKED", (865, H+90+40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 100, 0), 2)
            cv2.putText(FR, "--    DARK RED: HIGH RISK", (565, H+150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 150), 2)
            cv2.putText(FR, "--   RED: UNMASKED", (865, H+150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.putText(FR, "--      ORANGE: LOW RISK", (565, H+170),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 120, 255), 2)
            
            tot_str = "TOTAL COUNT:" + str(total_p)
            high_str = "HIGH RISK COUNT:" + str(high_risk_p)
            low_str = "LOW RISK COUNT:" + str(low_risk_p)
            safe_str = "SAFE COUNT:" + str(safe_p)
            masked_str="MASKED COUNT:" + str(masked_face_count)
            unmasked_str="UNMASKED COUNT:" + str(unmasked_face_count)
            unknown_str="UNKNOWN COUNT:" + str(total_p-masked_face_count-unmasked_face_count)
            cv2.putText(FR, tot_str, (1, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            cv2.putText(FR, safe_str, (160, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(FR, low_str, (310, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 120, 255), 2)
            cv2.putText(FR, high_str, (500, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 150), 2)
            cv2.putText(FR, masked_str, (700, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 100, 0), 2)
            cv2.putText(FR, unmasked_str, (880, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            cv2.putText(FR, unknown_str, (1080, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            if status[kk] == 1:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 150), 2)

            elif status[kk] == 0:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            else:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 120, 255), 2)

            kk += 1
        for h in close_pair:
            cv2.line(frame, tuple(h[0]), tuple(h[1]), (0, 0, 255), 2)
        for b in s_close_pair:
            cv2.line(frame, tuple(b[0]), tuple(b[1]), (0, 255, 255), 2)            

        # Put Bounding Boxes on Faces in the Frame
        # Green if Safe, Red if UnSafe
        for f in range(masked_face_count):
            a,b,c,d = masked_faces[f]
            cv2.rectangle(frame, (a, b), (c,d), (0,100,0), 2)

        for f in range(unmasked_face_count):
            a,b,c,d = unmasked_faces[f]
            cv2.rectangle(frame, (a, b), (c,d), (0,0,255), 2)

        FR[0:H, 0:W] = frame  
        frame = FR
    # Write Frame to the Output File
    
    # Save the Frame in frame_no.png format (If not required, comment the command below)
    # cv2.imwrite(BASE_PATH+"Results1/Frames/"+str(frame)+".jpg",img)

    # Use if you want to see Results Frame by Frame
        cv2_imshow(frame)
        cv2.waitKey(1)

    # Exit on Pressing Q Key
    # if cv2.waitKey(25) & 0xFF == ord('q'):
    #     break
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
        writer = cv2.VideoWriter(BASE_PATH+'Results/test.mp4', fourcc, 30,(frame.shape[1], frame.shape[0]), True)
    writer.write(frame)


# Release Streams
writer.release()
vs.release()
cv2.destroyAllWindows()

# Good to Go!
print("Done !")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2020-07-22 16:24:38--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M   455KB/s    in 75s     

2020-07-22 16:25:55 (459 KB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



In [3]:
%cd 'Social_Distancing_with_AI'

/content/drive/My Drive/Social_Distancing_with_AI


In [4]:
!git init

Reinitialized existing Git repository in /content/drive/My Drive/Social_Distancing_with_AI/.git/


In [ ]:
!git add .

In [ ]:
!git commit -m "Project for covid19"

In [ ]:
!git remote add origin https://github.com/jaskirat111/Social-Distancing-Analyser-and-Mask-Monitoring-AI-system-wrt-Covid-19.git

In [ ]:
!git push -u origin master